In [55]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn


In [56]:
url = "https://www.linkedin.com/jobs/search?keywords=Data%20&location=Finland%20sortBy%3DDD&geoId=100456013&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
no_of_jobs = 25

In [48]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome('/Users/admin/Documents/Data_Science_2021/Machine_Learning/Coursera_ML/chromedriver')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [49]:
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [ ]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')


# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section[1]/section[3]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    #/html/body/main/section[1]/section[3]/ul/li[1]/h3
    # Employment type
    #/html/body/main/section[1]/section[3]/ul/li[2]/h3
    type_xpath = '/html/body/main/section/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
   # function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    #job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    #functions.append(job_function)
    
    # Industries
    #industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    #industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    #industries.append(industry_type)
    
    x = x+1

In [54]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))

24
24
24
24
24
1
0


In [14]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        25 non-null     object
 1   Date          25 non-null     object
 2   Company Name  25 non-null     object
 3   Post          25 non-null     object
 4   Location      25 non-null     object
 5   Description   25 non-null     object
 6   Level         25 non-null     object
dtypes: object(7)
memory usage: 1.5+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level
0,2549025509,2021-05-14,Opset Oy Graphic,Business Data Analyst,"Helsinki, Southern Finland, Finland",Opset on OP Ryhmän pitkäaikainen kumppani ja t...,Temporary
1,2491424180,2021-03-12,Unity Graphic,Data Analyst - Helsinki,"Helsinki, Uusimaa, Finland",Role Description The Business Intelligence te...,Entry level
2,2511682839,2021-04-20,Supermetrics Graphic,Data Analyst,Helsinki Metropolitan Area,Direct message the job poster from Supermetric...,Mid-Senior level
3,2417294106,2021-04-21,Unity Technologies Graphic,Data Analyst,"Helsinki, Southern Finland, Finland",Direct message the job poster from Unity Techn...,Entry level
4,2542981153,2021-05-11,CGI Graphic,Data Analyst,"Turku, West and Inner Finland, Finland",Job Description Job Description Data Analyst...,Entry level


In [15]:
job_data.to_csv('LinkedIn Job Data.csv', index=0)

In [16]:
job_data.shape

(25, 7)

In [17]:
job_data.columns.to_list()

['Job ID', 'Date', 'Company Name', 'Post', 'Location', 'Description', 'Level']